In [1]:
import imaplib
import email
from email.header import decode_header
import os

In [3]:

# Email Credentials
EMAIL = 'a0049632@gmail.com'  # Replace with your email
PASSWORD = 'a0049632abhi12!@'        # Replace with your password
IMAP_SERVER = 'imap.gmail.com'    # IMAP server for Gmail

In [5]:

# Function to connect to IMAP and download attachments
def download_attachment_from_email():
    try:
        # Connect to the IMAP server
        mail = imaplib.IMAP4_SSL(IMAP_SERVER)
        mail.login(EMAIL, PASSWORD)
        mail.select("inbox")  # Access the inbox

        # Search for all unread emails
        status, messages = mail.search(None, 'UNSEEN')
        mail_ids = messages[0].split()

        for mail_id in mail_ids:
            # Fetch the email
            _, msg_data = mail.fetch(mail_id, '(RFC822)')
            for response_part in msg_data:
                if isinstance(response_part, tuple):
                    # Parse the email content
                    msg = email.message_from_bytes(response_part[1])
                    subject, encoding = decode_header(msg["Subject"])[0]
                    if isinstance(subject, bytes):
                        subject = subject.decode(encoding if encoding else "utf-8")
                    print(f"Processing Email: {subject}")

                    # Check for attachments
                    for part in msg.walk():
                        content_type = part.get_content_type()
                        content_disposition = str(part.get("Content-Disposition"))

                        if "attachment" in content_disposition:
                            filename = part.get_filename()
                            if filename:
                                filename = decode_header(filename)[0][0]
                                if isinstance(filename, bytes):
                                    filename = filename.decode("utf-8")

                                # Save the attachment
                                filepath = os.path.join("attachments", filename)
                                if not os.path.isdir("attachments"):
                                    os.mkdir("attachments")

                                with open(filepath, "wb") as f:
                                    f.write(part.get_payload(decode=True))
                                print(f"Attachment Saved: {filename}")
                                return filepath
        print("No attachments found.")
        return None
    except Exception as e:
        print(f"Error: {str(e)}")
        return None

# Main Execution
if __name__ == "__main__":
    attachment_path = download_attachment_from_email()
    if attachment_path:
        print(f"Attachment downloaded at: {attachment_path}")
    else:
        print("No new email with attachments.")


Error: [WinError 10054] An existing connection was forcibly closed by the remote host
No new email with attachments.
